In [6]:
import numpy as numpy
import torch 
from model_dynamic_CP import LDS_dynammic_CP
import utils
import tqdm
import yaml

path = "base.yaml"
with open(path, "r") as f:
    config = yaml.safe_load(f)

# data_file = '../processed_data/beijing_15k.npy'
data_file = '../processed_data/mvlens_10k.npy'

hyper_dict = utils.make_hyper_dict(config)
data_dict = utils.make_data_dict(hyper_dict,data_file)

model = LDS_dynammic_CP(hyper_dict, data_dict)

use device: cpu


In [16]:
# mode = 0
# model.LDS_list[mode].reset_list()

# for k in range(model.N_time):

#     # update model.msg_U_M, model.msg_U_V : compute Ez, Ez2, get \beta, S, emseble to long vec msg_U
#     model.msg_update_U(mode, k)

#     # Kalman Filter: predict and update
#     model.LDS_list[mode].filter_predict(ind=k)
#     model.LDS_list[mode].filter_update(y=model.msg_U_M, R=model.msg_U_V)

# model.LDS_list[mode].smooth()
# model.post_update_U(mode)

In [7]:
for i in tqdm.tqdm(range(30)):

    for mode in range(model.nmods):

        model.LDS_list[mode].reset_list()
        
        for k in range(model.N_time):

            # update model.msg_U_M, model.msg_U_V : compute Ez, Ez2, get \beta, S, emseble to long vec msg_U
            model.msg_update_U(mode, k)

            # Kalman Filter: predict and update
            model.LDS_list[mode].filter_predict(ind=k)
            model.LDS_list[mode].filter_update(y=model.msg_U_M, R=model.msg_U_V)

        model.LDS_list[mode].smooth()

        model.post_update_U(mode)

    test_result = model.model_test(model.ind_te, model.y_te, model.test_time_ind).cpu().numpy()

    print("epoch:", i, 'test-rmse:',test_result)

  0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
model.post_U_m[2]

tensor([[[[ 5.1251e-03,  5.0288e-03,  4.9276e-03,  ..., -4.9635e-04,
           -3.7468e-04, -2.5580e-04]],

         [[ 8.8006e-03,  8.6875e-03,  8.5687e-03,  ...,  2.4943e-04,
            1.3882e-04,  3.0789e-05]]],


        [[[ 6.2479e-03,  6.1392e-03,  6.0250e-03,  ...,  1.5002e-04,
            1.5778e-04,  1.6534e-04]],

         [[ 6.8264e-03,  6.7113e-03,  6.5906e-03,  ..., -2.8678e-04,
           -3.3470e-04, -3.8145e-04]]],


        [[[ 6.5536e-03,  6.4386e-03,  6.3181e-03,  ...,  2.8377e-04,
            2.9791e-04,  3.1167e-04]],

         [[ 7.1047e-03,  6.9885e-03,  6.8667e-03,  ..., -2.2384e-04,
           -2.5410e-04, -2.8363e-04]]],


        ...,


        [[[ 5.4756e-03,  5.3760e-03,  5.2714e-03,  ...,  7.3272e-05,
            1.8894e-04,  3.0186e-04]],

         [[ 6.7351e-03,  6.6200e-03,  6.4993e-03,  ..., -3.7380e-04,
           -4.6509e-04, -5.5417e-04]]],


        [[[ 5.5856e-03,  5.4909e-03,  5.3913e-03,  ...,  6.2971e-05,
            4.9768e-05,  3.6863e-05]

In [11]:
model.msg_U_M


tensor([[ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 1.1809e-05],
        [ 4.0384e-06],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [-1.0967e-06],
        [ 8.2535e-07],
        [ 0.0000e+00],
        [ 0.0000e+00]], device='cuda:0')

In [11]:
test_result.cpu().numpy()

array(0.95424135)

In [7]:
import numpy as np
a = np.zeros((2,2))
len(a.shape)

2